<a href="https://colab.research.google.com/github/danielpatrickhug/kernel-trick/blob/main/diagonal_elements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
# Usage
m = 5
n = 3
embedding_matrix = np.random.lognormal(size=(m, n))

In [55]:
def check_symmetry(kernel_matrix):
    # Check if the input matrix is square
    if kernel_matrix.shape[0] != kernel_matrix.shape[1]:
        raise ValueError("The input kernel matrix must be square.")

    # Check if the difference between the matrix and its transpose is close to zero
    is_symmetric = np.allclose(kernel_matrix, kernel_matrix.T, atol=1e-8)

    return is_symmetric


In [56]:
def is_positive_semi_definite(kernel_matrix):
    # Check if the input matrix is square
    if kernel_matrix.shape[0] != kernel_matrix.shape[1]:
        raise ValueError("The input kernel matrix must be square.")
    
    # Compute the eigenvalues of the kernel matrix
    eigenvalues = np.linalg.eigvalsh(kernel_matrix)
    
    # Check if all eigenvalues are non-negative (within a small tolerance)
    is_psd = np.all(eigenvalues >= -1e-8)
    
    return is_psd

In [57]:

def gram_matrix(embedding_matrix):
    # Compute the kernel matrix using matrix multiplication. AAT
    kernel_matrix = np.matmul(embedding_matrix, embedding_matrix.T)
    return kernel_matrix


def covariance_matrix(embedding_matrix):
    # Compute the kernel matrix using matrix multiplication. AAT
    m, n = embedding_matrix.shape  # number of samples (m) and number of features (n)
    mean_centered_matrix = embedding_matrix - np.mean(embedding_matrix, axis=0)  # centering the data
    cov_matrix = np.matmul(mean_centered_matrix.T, mean_centered_matrix) / (m - 1)  # calculating covariance matrix

    return cov_matrix



## Gram and Cov Diagonals

In [58]:
gram = gram_matrix(embedding_matrix)
print(gram)
print(f'is Symmetric? {check_symmetry(gram)}')
print(f'is Positive Semi Definite? {is_positive_semi_definite(gram)}')
cov = covariance_matrix(embedding_matrix)
print(cov )
print(f'is Symmetric? {check_symmetry(cov)}')
print(f'is Positive Semi Definite? {is_positive_semi_definite(cov)}')


[[ 9.4100522   9.97024925  9.45793403  7.33081966  2.43664298]
 [ 9.97024925 69.85482313 10.26042386  7.6475513   9.4062474 ]
 [ 9.45793403 10.26042386 13.70016037 15.46430788  3.84780709]
 [ 7.33081966  7.6475513  15.46430788 21.34539717  4.53217781]
 [ 2.43664298  9.4062474   3.84780709  4.53217781  1.86486849]]
is Symmetric? True
is Positive Semi Definite? True
[[ 1.02154844 -0.24401727 -0.35568195]
 [-0.24401727  2.56929202 -2.39676176]
 [-0.35568195 -2.39676176 11.60880369]]
is Symmetric? True
is Positive Semi Definite? True


In [59]:
lower_tri_gram = np.tril(gram)
upper_tri_cov = np.triu(cov)
transformed_lower = np.triu(gram) + np.triu(gram, 1).T
transformed_upper = np.tril(cov) + np.tril(cov, -1).T
gram_diagonal = list(np.diag(transformed_lower))
cov_diagonal = list(np.diag(transformed_upper))

In [60]:
np.diag(lower_tri_gram) == gram_diagonal

array([ True,  True,  True,  True,  True])

In [61]:
np.diag(upper_tri_cov) == cov_diagonal

array([ True,  True,  True])

In [62]:
from scipy.stats import spearmanr, kendalltau

In [63]:
aligned_gram_diagonal = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, m), gram_diagonal)

correlation, p_value = pearsonr(aligned_gram_diagonal, cov_diagonal)
print("Correlation coefficient:", correlation)
print("P-value:", p_value)
# Calculate Spearman correlation coefficient and p-value
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_diagonal, cov_diagonal)
print("Spearman Correlation coefficient:", spearman_corr)
print("P-value:", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_diagonal, cov_diagonal)
print("Kendall Correlation coefficient:", kendall_corr)
print("P-value:", kendall_pvalue)

Correlation coefficient: -0.8766837372950342
P-value: 0.3195013774579005
Spearman Correlation coefficient: -0.5
P-value: 0.6666666666666667
Kendall Correlation coefficient: -0.33333333333333337
P-value: 1.0


In [64]:
from scipy.interpolate import CubicSpline
from scipy.stats import pearsonr
m = len(gram_diagonal)
n = len(cov_diagonal)

# Create cubic spline objects
gram_spline = CubicSpline(np.linspace(0, 1, m), gram_diagonal)
cov_spline = CubicSpline(np.linspace(0, 1, n), cov_diagonal)

# Evaluate cubic splines at desired points
aligned_gram_diagonal = gram_spline(np.linspace(0, 1, n))

correlation, p_value = pearsonr(aligned_gram_diagonal, cov_diagonal)
print("Correlation coefficient:", correlation)
print("P-value:", p_value)
# Calculate Spearman correlation coefficient and p-value
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_diagonal, cov_diagonal)
print("Spearman Correlation coefficient:", spearman_corr)
print("P-value:", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_diagonal, cov_diagonal)
print("Kendall Correlation coefficient:", kendall_corr)
print("P-value:", kendall_pvalue)

Correlation coefficient: -0.8766837372950342
P-value: 0.3195013774579005
Spearman Correlation coefficient: -0.5
P-value: 0.6666666666666667
Kendall Correlation coefficient: -0.33333333333333337
P-value: 1.0


In [65]:
aligned_cov_diagonal = np.interp(np.linspace(0, 1, m), np.linspace(0, 1, n), cov_diagonal)


correlation, p_value = pearsonr(gram_diagonal, aligned_cov_diagonal)
print("Correlation coefficient:", correlation)
print("P-value:", p_value)
# Calculate Spearman correlation coefficient and p-value
spearman_corr, spearman_pvalue = spearmanr(gram_diagonal, aligned_cov_diagonal)
print("Spearman Correlation coefficient:", spearman_corr)
print("P-value:", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value
kendall_corr, kendall_pvalue = kendalltau(gram_diagonal, aligned_cov_diagonal)
print("Kendall Correlation coefficient:", kendall_corr)
print("P-value:", kendall_pvalue)

Correlation coefficient: -0.4483994334181218
P-value: 0.44883484783758915
Spearman Correlation coefficient: -0.3
P-value: 0.6238376647810728
Kendall Correlation coefficient: -0.19999999999999998
P-value: 0.8166666666666667


In [66]:
# Evaluate cubic splines at desired points
aligned_cov_diagonal = cov_spline(np.linspace(0, 1, m))

correlation, p_value = pearsonr(gram_diagonal, aligned_cov_diagonal)
print("Correlation coefficient:", correlation)
print("P-value:", p_value)
# Calculate Spearman correlation coefficient and p-value
spearman_corr, spearman_pvalue = spearmanr(gram_diagonal, aligned_cov_diagonal)
print("Spearman Correlation coefficient:", spearman_corr)
print("P-value:", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value
kendall_corr, kendall_pvalue = kendalltau(gram_diagonal, aligned_cov_diagonal)
print("Kendall Correlation coefficient:", kendall_corr)
print("P-value:", kendall_pvalue)

Correlation coefficient: -0.5271557630167503
P-value: 0.3613363857478348
Spearman Correlation coefficient: -0.6
P-value: 0.28475697986529375
Kendall Correlation coefficient: -0.39999999999999997
P-value: 0.48333333333333334


## Graph laplacian Diagonals

In [67]:
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.interpolate import CubicSpline

# Calculate the diagonal of the graph Laplacian for the Gram matrix
gram_Laplacian_diag = np.sum(gram, axis=1) - np.diag(gram)
print("Diagonal of the graph Laplacian of gram_matrix:")
print(gram_Laplacian_diag)

# Calculate the diagonal of the graph Laplacian for the Covariance matrix
cov_Laplacian_diag = np.sum(cov, axis=1) - np.diag(cov)
print("Diagonal of the graph Laplacian of Cov_matrix:")
print(cov_Laplacian_diag)

n = len(cov_Laplacian_diag)
m = len(gram_Laplacian_diag)

# Aligning Gram Laplacian diagonal to Covariance matrix size
aligned_gram_Laplacian_diag = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, m), gram_Laplacian_diag)

# Calculate Pearson correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
correlation, p_value = pearsonr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals):", correlation)
print("P-value (Gram to Cov, Laplacian diagonals):", p_value)

# Calculate Spearman correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals):", spearman_corr)
print("P-value (Gram to Cov, Laplacian diagonals):", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Kendall Correlation coefficient (Gram to Cov, Laplacian diagonals):", kendall_corr)
print("P-value (Gram to Cov, Laplacian diagonals):", kendall_pvalue)

# Create cubic spline objects
gram_spline = CubicSpline(np.linspace(0, 1, m), gram_Laplacian_diag)
cov_spline = CubicSpline(np.linspace(0, 1, n), cov_Laplacian_diag)

# Evaluate cubic splines at desired points
aligned_gram_Laplacian_diag = gram_spline(np.linspace(0, 1, n))

# Calculate Pearson correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
correlation, p_value = pearsonr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines):", correlation)
print("P-value (Gram to Cov, Laplacian diagonals using cubic splines):", p_value)

# Calculate Spearman correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines):", spearman_corr)
print("P-value (Gram to Cov, Laplacian diagonals using cubic splines):", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Kendall Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines):", kendall_corr)
print("P-value (Gram to Cov, Laplacian diagonals using cubic splines):", kendall_pvalue)

# Aligning Covariance Laplacian diagonal to Gram matrix size
aligned_cov_Laplacian_diag = np.interp(np.linspace(0, 1, m), np.linspace(0, 1, n), cov_Laplacian_diag)

# Calculate Pearson correlation coefficient and p-value between Gram Laplacian diagonal and aligned Covariance matrix Laplacian diagonal
correlation, p_value = pearsonr(gram_Laplacian_diag, aligned_cov_Laplacian_diag)
print("Pearson Correlation coefficient (Cov to Gram, Laplacian diagonals):", correlation)
print("P-value (Cov to Gram, Laplacian diagonals):", p_value)

# Calculate Spearman correlation coefficient and p-value between Gram Laplacian diagonal and aligned Covariance matrix Laplacian diagonal
spearman_corr, spearman_pvalue = spearmanr(gram_Laplacian_diag, aligned_cov_Laplacian_diag)
print("Spearman Correlation coefficient (Cov to Gram, Laplacian diagonals):", spearman_corr)
print("P-value (Cov to Gram, Laplacian diagonals):", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value between Gram Laplacian diagonal and aligned Covariance matrix Laplacian diagonal
kendall_corr, kendall_pvalue = kendalltau(gram_Laplacian_diag, aligned_cov_Laplacian_diag)
print("Kendall Correlation coefficient (Cov to Gram, Laplacian diagonals):", kendall_corr)
print("P-value (Cov to Gram, Laplacian diagonals):", kendall_pvalue)



Diagonal of the graph Laplacian of gram_matrix:
[29.19564591 37.2844718  39.03047286 34.97485664 20.22287527]
Diagonal of the graph Laplacian of Cov_matrix:
[-0.59969921 -2.64077903 -2.75244371]
Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals): 0.019626651606323098
P-value (Gram to Cov, Laplacian diagonals): 0.9875044832109366
Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals): 0.5
P-value (Gram to Cov, Laplacian diagonals): 0.6666666666666667
Kendall Correlation coefficient (Gram to Cov, Laplacian diagonals): 0.33333333333333337
P-value (Gram to Cov, Laplacian diagonals): 1.0
Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines): 0.019626651606323098
P-value (Gram to Cov, Laplacian diagonals using cubic splines): 0.9875044832109366
Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines): 0.5
P-value (Gram to Cov, Laplacian diagonals using cubic splines): 0.6666666666666667
Kendall Corre

## Singular Value Diagonals

In [68]:
import numpy as np
from scipy.stats import pearsonr
from scipy.interpolate import CubicSpline

# Gram matrix singular value diagonal
gram_svd_diag = np.linalg.svd(gram, compute_uv=False)

# Covariance matrix singular value diagonal
cov_svd_diag = np.linalg.svd(cov, compute_uv=False)

n = len(cov_svd_diag)
m = len(gram_svd_diag)

# Aligning Gram matrix singular value diagonal to Covariance matrix size
aligned_gram_svd_diag = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, m), gram_svd_diag)

# Calculating Pearson correlation coefficient and p-value between aligned Gram matrix and Covariance matrix singular value diagonals
correlation, p_value = pearsonr(aligned_gram_svd_diag, cov_svd_diag)
print("Pearson Correlation coefficient (Gram to Cov):", correlation)
print("P-value (Gram to Cov):", p_value)

# Calculating Spearman correlation coefficient and p-value between aligned Gram matrix and Covariance matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_svd_diag, cov_svd_diag)
print("Spearman Correlation coefficient (Gram to Cov):", spearman_corr)
print("P-value (Gram to Cov):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value between aligned Gram matrix and Covariance matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_svd_diag, cov_svd_diag)
print("Kendall Correlation coefficient (Gram to Cov):", kendall_corr)
print("P-value (Gram to Cov):", kendall_pvalue)

# Aligning Covariance matrix singular value diagonal to Gram matrix size
aligned_cov_svd_diag = np.interp(np.linspace(0, 1, m), np.linspace(0, 1, n), cov_svd_diag)

# Calculating Pearson correlation coefficient and p-value between Covariance matrix and aligned Gram matrix singular value diagonals
correlation, p_value = pearsonr(gram_svd_diag, aligned_cov_svd_diag)
print("Pearson Correlation coefficient (Cov to Gram):", correlation)
print("P-value (Cov to Gram):", p_value)

# Calculating Spearman correlation coefficient and p-value between Covariance matrix and aligned Gram matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(gram_svd_diag, aligned_cov_svd_diag)
print("Spearman Correlation coefficient (Cov to Gram):", spearman_corr)
print("P-value (Cov to Gram):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value between Covariance matrix and aligned Gram matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(gram_svd_diag, aligned_cov_svd_diag)
print("Kendall Correlation coefficient (Cov to Gram):", kendall_corr)
print("P-value (Cov to Gram):", kendall_pvalue)

# Create cubic spline objects
gram_svd_diag_spline = CubicSpline(np.linspace(0, 1, m), gram_svd_diag)
cov_svd_diag_spline = CubicSpline(np.linspace(0, 1, n), cov_svd_diag)

# Evaluate cubic splines at desired points
aligned_gram_svd_diag_spline = gram_svd_diag_spline(np.linspace(0, 1, n))
aligned_cov_svd_diag_spline = cov_svd_diag_spline(np.linspace(0, 1, m))

# Calculating Pearson correlation coefficient and p-value (using cubic spline interpolation) between aligned Gram matrix and Covariance matrix singular value diagonals
correlation, p_value = pearsonr(aligned_gram_svd_diag_spline, cov_svd_diag)
print("Pearson Correlation coefficient (using cubic spline interpolation, Gram to Cov):", correlation)
print("P-value (using cubic spline interpolation, Gram to Cov):", p_value)

# Calculating Spearman correlation coefficient and p-value (using cubic spline interpolation) between aligned Gram matrix and Covariance matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_svd_diag_spline, cov_svd_diag)
print("Spearman Correlation coefficient (using cubic spline interpolation, Gram to Cov):", spearman_corr)
print("P-value (using cubic spline interpolation, Gram to Cov):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value (using cubic spline interpolation) between aligned Gram matrix and Covariance matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_svd_diag_spline, cov_svd_diag)
print("Kendall Correlation coefficient (using cubic spline interpolation, Gram to Cov):", kendall_corr)
print("P-value (using cubic spline interpolation, Gram to Cov):", kendall_pvalue)

# Calculating Pearson correlation coefficient and p-value (using cubic spline interpolation) between Covariance matrix and aligned Gram matrix singular value diagonals
correlation, p_value = pearsonr(gram_svd_diag, aligned_cov_svd_diag_spline)
print("Pearson Correlation coefficient (using cubic spline interpolation, Cov to Gram):", correlation)
print("P-value (using cubic spline interpolation, Cov to Gram):", p_value)

# Calculating Spearman correlation coefficient and p-value (using cubic spline interpolation) between Covariance matrix and aligned Gram matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(gram_svd_diag, aligned_cov_svd_diag_spline)
print("Spearman Correlation coefficient (using cubic spline interpolation, Cov to Gram):", spearman_corr)
print("P-value (using cubic spline interpolation, Cov to Gram):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value (using cubic spline interpolation) between Covariance matrix and aligned Gram matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(gram_svd_diag, aligned_cov_svd_diag_spline)
print("Kendall Correlation coefficient (using cubic spline interpolation, Cov to Gram):", kendall_corr)
print("P-value (using cubic spline interpolation, Cov to Gram):", kendall_pvalue)


Pearson Correlation coefficient (Gram to Cov): 0.9996597710163792
P-value (Gram to Cov): 0.01660707277254271
Spearman Correlation coefficient (Gram to Cov): 1.0
P-value (Gram to Cov): 0.0
Kendall Correlation coefficient (Gram to Cov): 1.0
P-value (Gram to Cov): 0.3333333333333333
Pearson Correlation coefficient (Cov to Gram): 0.9913288056410701
P-value (Cov to Gram): 0.0009680249576038099
Spearman Correlation coefficient (Cov to Gram): 0.9999999999999999
P-value (Cov to Gram): 1.4042654220543672e-24
Kendall Correlation coefficient (Cov to Gram): 0.9999999999999999
P-value (Cov to Gram): 0.016666666666666666
Pearson Correlation coefficient (using cubic spline interpolation, Gram to Cov): 0.9996597710163792
P-value (using cubic spline interpolation, Gram to Cov): 0.01660707277254271
Spearman Correlation coefficient (using cubic spline interpolation, Gram to Cov): 1.0
P-value (using cubic spline interpolation, Gram to Cov): 0.0
Kendall Correlation coefficient (using cubic spline interpola

## Message Passing

In [72]:
import numpy as np

def message_passing(embeddings, gram_matrix, num_iterations):
    num_nodes = embeddings.shape[0]
    
    # Initialize message passed embeddings
    message_passed_embeddings = np.copy(embeddings)
    
    for _ in range(num_iterations):
        # Compute messages from neighbors
        messages = np.matmul(gram_matrix, message_passed_embeddings)
        
        # Update embeddings by aggregating messages
        message_passed_embeddings = embeddings + messages / num_nodes
    
    # Compute new Gram matrix
    new_gram_matrix = np.matmul(message_passed_embeddings, message_passed_embeddings.T)
    
    # Compute new Covariance matrix
    new_cov_matrix = np.cov(message_passed_embeddings.T)
    return message_passed_embeddings, new_gram_matrix, new_cov_matrix
message_passed_embeddings, new_gram_matrix, new_cov_matrix = message_passing(embedding_matrix, gram, 2)

print("Original Embeddings")
print(embedding_matrix)
print("Message Passed Embeddings:")
print(message_passed_embeddings)
print("New Gram Matrix:")
print(new_gram_matrix)
print("New Covariance Matrix:")
print(new_cov_matrix)

Original Embeddings
[[2.74735851 1.19526749 0.6583381 ]
 [1.18167042 1.10928083 8.19926669]
 [1.96805548 3.08612526 0.5502262 ]
 [0.62550376 4.57208525 0.22400607]
 [0.25523266 0.90796738 0.98758292]]
Message Passed Embeddings:
[[ 146.11259439  223.2737056   367.31390398]
 [ 501.38603287  654.69529697 1932.15558499]
 [ 188.48285796  318.16044658  426.22054883]
 [ 190.80421318  350.86127021  381.14198316]
 [  90.88089426  136.04777261  289.32702796]]
New Gram Matrix:
[[ 206119.54190893  929142.67006574  255133.31500531  246215.74436496
   149928.57373667]
 [ 929142.67006574 4413239.09045334 1126325.23433669 1061799.40243521
   693661.08088614]
 [ 255133.31500531 1126325.23433669  318415.81375692  310044.04707182
   183731.63542247]
 [ 246215.74436496 1061799.40243521  310044.04707182  304779.09002565
   175349.02904828]
 [ 149928.57373667  693661.08088614  183731.63542247  175349.02904828
   110478.462481  ]]
New Covariance Matrix:
[[ 25773.12354514  30985.18520953 110655.84276343]
 [ 3

## Graph laplacian diagonals of the Message passed embedding matrix

In [73]:
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.interpolate import CubicSpline

# Calculate the diagonal of the graph Laplacian for the Gram matrix
gram_Laplacian_diag = np.sum(new_gram_matrix, axis=1) - np.diag(new_gram_matrix)
print("Diagonal of the graph Laplacian of gram_matrix:")
print(gram_Laplacian_diag)

# Calculate the diagonal of the graph Laplacian for the Covariance matrix
cov_Laplacian_diag = np.sum(new_cov_matrix, axis=1) - np.diag(new_cov_matrix)
print("Diagonal of the graph Laplacian of Cov_matrix:")
print(cov_Laplacian_diag)

n = len(cov_Laplacian_diag)
m = len(gram_Laplacian_diag)

# Aligning Gram Laplacian diagonal to Covariance matrix size
aligned_gram_Laplacian_diag = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, m), gram_Laplacian_diag)

# Calculate Pearson correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
correlation, p_value = pearsonr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals):", correlation)
print("P-value (Gram to Cov, Laplacian diagonals):", p_value)

# Calculate Spearman correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals):", spearman_corr)
print("P-value (Gram to Cov, Laplacian diagonals):", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Kendall Correlation coefficient (Gram to Cov, Laplacian diagonals):", kendall_corr)
print("P-value (Gram to Cov, Laplacian diagonals):", kendall_pvalue)

# Create cubic spline objects
gram_spline = CubicSpline(np.linspace(0, 1, m), gram_Laplacian_diag)
cov_spline = CubicSpline(np.linspace(0, 1, n), cov_Laplacian_diag)

# Evaluate cubic splines at desired points
aligned_gram_Laplacian_diag = gram_spline(np.linspace(0, 1, n))

# Calculate Pearson correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
correlation, p_value = pearsonr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines):", correlation)
print("P-value (Gram to Cov, Laplacian diagonals using cubic splines):", p_value)

# Calculate Spearman correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines):", spearman_corr)
print("P-value (Gram to Cov, Laplacian diagonals using cubic splines):", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value between aligned Gram Laplacian diagonal and Covariance matrix Laplacian diagonal
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_Laplacian_diag, cov_Laplacian_diag)
print("Kendall Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines):", kendall_corr)
print("P-value (Gram to Cov, Laplacian diagonals using cubic splines):", kendall_pvalue)

# Aligning Covariance Laplacian diagonal to Gram matrix size
aligned_cov_Laplacian_diag = np.interp(np.linspace(0, 1, m), np.linspace(0, 1, n), cov_Laplacian_diag)

# Calculate Pearson correlation coefficient and p-value between Gram Laplacian diagonal and aligned Covariance matrix Laplacian diagonal
correlation, p_value = pearsonr(gram_Laplacian_diag, aligned_cov_Laplacian_diag)
print("Pearson Correlation coefficient (Cov to Gram, Laplacian diagonals):", correlation)
print("P-value (Cov to Gram, Laplacian diagonals):", p_value)

# Calculate Spearman correlation coefficient and p-value between Gram Laplacian diagonal and aligned Covariance matrix Laplacian diagonal
spearman_corr, spearman_pvalue = spearmanr(gram_Laplacian_diag, aligned_cov_Laplacian_diag)
print("Spearman Correlation coefficient (Cov to Gram, Laplacian diagonals):", spearman_corr)
print("P-value (Cov to Gram, Laplacian diagonals):", spearman_pvalue)

# Calculate Kendall correlation coefficient and p-value between Gram Laplacian diagonal and aligned Covariance matrix Laplacian diagonal
kendall_corr, kendall_pvalue = kendalltau(gram_Laplacian_diag, aligned_cov_Laplacian_diag)
print("Kendall Correlation coefficient (Cov to Gram, Laplacian diagonals):", kendall_corr)
print("P-value (Cov to Gram, Laplacian diagonals):", kendall_pvalue)



Diagonal of the graph Laplacian of gram_matrix:
[1580420.30317267 3810928.38772378 1875234.2318363  1793408.22292026
 1202670.31909356]
Diagonal of the graph Laplacian of Cov_matrix:
[141641.02797297 159112.23729423 238782.89484813]
Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals): -0.812697880284875
P-value (Gram to Cov, Laplacian diagonals): 0.39599594021986334
Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals): -0.5
P-value (Gram to Cov, Laplacian diagonals): 0.6666666666666667
Kendall Correlation coefficient (Gram to Cov, Laplacian diagonals): -0.33333333333333337
P-value (Gram to Cov, Laplacian diagonals): 1.0
Pearson Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines): -0.812697880284875
P-value (Gram to Cov, Laplacian diagonals using cubic splines): 0.39599594021986334
Spearman Correlation coefficient (Gram to Cov, Laplacian diagonals using cubic splines): -0.5
P-value (Gram to Cov, Laplacian diagonals using cubic spl

## Singular value diagonals of the Message passed embedding matrix

In [74]:
import numpy as np
from scipy.stats import pearsonr
from scipy.interpolate import CubicSpline

# Gram matrix singular value diagonal
gram_svd_diag = np.linalg.svd(new_gram_matrix, compute_uv=False)

# Covariance matrix singular value diagonal
cov_svd_diag = np.linalg.svd(new_cov_matrix, compute_uv=False)

n = len(cov_svd_diag)
m = len(gram_svd_diag)

# Aligning Gram matrix singular value diagonal to Covariance matrix size
aligned_gram_svd_diag = np.interp(np.linspace(0, 1, n), np.linspace(0, 1, m), gram_svd_diag)

# Calculating Pearson correlation coefficient and p-value between aligned Gram matrix and Covariance matrix singular value diagonals
correlation, p_value = pearsonr(aligned_gram_svd_diag, cov_svd_diag)
print("Pearson Correlation coefficient (Gram to Cov):", correlation)
print("P-value (Gram to Cov):", p_value)

# Calculating Spearman correlation coefficient and p-value between aligned Gram matrix and Covariance matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_svd_diag, cov_svd_diag)
print("Spearman Correlation coefficient (Gram to Cov):", spearman_corr)
print("P-value (Gram to Cov):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value between aligned Gram matrix and Covariance matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_svd_diag, cov_svd_diag)
print("Kendall Correlation coefficient (Gram to Cov):", kendall_corr)
print("P-value (Gram to Cov):", kendall_pvalue)

# Aligning Covariance matrix singular value diagonal to Gram matrix size
aligned_cov_svd_diag = np.interp(np.linspace(0, 1, m), np.linspace(0, 1, n), cov_svd_diag)

# Calculating Pearson correlation coefficient and p-value between Covariance matrix and aligned Gram matrix singular value diagonals
correlation, p_value = pearsonr(gram_svd_diag, aligned_cov_svd_diag)
print("Pearson Correlation coefficient (Cov to Gram):", correlation)
print("P-value (Cov to Gram):", p_value)

# Calculating Spearman correlation coefficient and p-value between Covariance matrix and aligned Gram matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(gram_svd_diag, aligned_cov_svd_diag)
print("Spearman Correlation coefficient (Cov to Gram):", spearman_corr)
print("P-value (Cov to Gram):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value between Covariance matrix and aligned Gram matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(gram_svd_diag, aligned_cov_svd_diag)
print("Kendall Correlation coefficient (Cov to Gram):", kendall_corr)
print("P-value (Cov to Gram):", kendall_pvalue)

# Create cubic spline objects
gram_svd_diag_spline = CubicSpline(np.linspace(0, 1, m), gram_svd_diag)
cov_svd_diag_spline = CubicSpline(np.linspace(0, 1, n), cov_svd_diag)

# Evaluate cubic splines at desired points
aligned_gram_svd_diag_spline = gram_svd_diag_spline(np.linspace(0, 1, n))
aligned_cov_svd_diag_spline = cov_svd_diag_spline(np.linspace(0, 1, m))

# Calculating Pearson correlation coefficient and p-value (using cubic spline interpolation) between aligned Gram matrix and Covariance matrix singular value diagonals
correlation, p_value = pearsonr(aligned_gram_svd_diag_spline, cov_svd_diag)
print("Pearson Correlation coefficient (using cubic spline interpolation, Gram to Cov):", correlation)
print("P-value (using cubic spline interpolation, Gram to Cov):", p_value)

# Calculating Spearman correlation coefficient and p-value (using cubic spline interpolation) between aligned Gram matrix and Covariance matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(aligned_gram_svd_diag_spline, cov_svd_diag)
print("Spearman Correlation coefficient (using cubic spline interpolation, Gram to Cov):", spearman_corr)
print("P-value (using cubic spline interpolation, Gram to Cov):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value (using cubic spline interpolation) between aligned Gram matrix and Covariance matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(aligned_gram_svd_diag_spline, cov_svd_diag)
print("Kendall Correlation coefficient (using cubic spline interpolation, Gram to Cov):", kendall_corr)
print("P-value (using cubic spline interpolation, Gram to Cov):", kendall_pvalue)

# Calculating Pearson correlation coefficient and p-value (using cubic spline interpolation) between Covariance matrix and aligned Gram matrix singular value diagonals
correlation, p_value = pearsonr(gram_svd_diag, aligned_cov_svd_diag_spline)
print("Pearson Correlation coefficient (using cubic spline interpolation, Cov to Gram):", correlation)
print("P-value (using cubic spline interpolation, Cov to Gram):", p_value)

# Calculating Spearman correlation coefficient and p-value (using cubic spline interpolation) between Covariance matrix and aligned Gram matrix singular value diagonals
spearman_corr, spearman_pvalue = spearmanr(gram_svd_diag, aligned_cov_svd_diag_spline)
print("Spearman Correlation coefficient (using cubic spline interpolation, Cov to Gram):", spearman_corr)
print("P-value (using cubic spline interpolation, Cov to Gram):", spearman_pvalue)

# Calculating Kendall correlation coefficient and p-value (using cubic spline interpolation) between Covariance matrix and aligned Gram matrix singular value diagonals
kendall_corr, kendall_pvalue = kendalltau(gram_svd_diag, aligned_cov_svd_diag_spline)
print("Kendall Correlation coefficient (using cubic spline interpolation, Cov to Gram):", kendall_corr)
print("P-value (using cubic spline interpolation, Cov to Gram):", kendall_pvalue)


Pearson Correlation coefficient (Gram to Cov): 0.9999596961584085
P-value (Gram to Cov): 0.005715704937585869
Spearman Correlation coefficient (Gram to Cov): 1.0
P-value (Gram to Cov): 0.0
Kendall Correlation coefficient (Gram to Cov): 1.0
P-value (Gram to Cov): 0.3333333333333333
Pearson Correlation coefficient (Cov to Gram): 0.8807073434866745
P-value (Cov to Gram): 0.04856535031550231
Spearman Correlation coefficient (Cov to Gram): 0.9999999999999999
P-value (Cov to Gram): 1.4042654220543672e-24
Kendall Correlation coefficient (Cov to Gram): 0.9999999999999999
P-value (Cov to Gram): 0.016666666666666666
Pearson Correlation coefficient (using cubic spline interpolation, Gram to Cov): 0.9999596961584085
P-value (using cubic spline interpolation, Gram to Cov): 0.005715704937585869
Spearman Correlation coefficient (using cubic spline interpolation, Gram to Cov): 1.0
P-value (using cubic spline interpolation, Gram to Cov): 0.0
Kendall Correlation coefficient (using cubic spline interpola